In [1]:
import os
import uuid
import json
import logging
from dotenv import load_dotenv

from dapr_agents import OpenAIChatClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from dapr_agents.tool import AgentTool

logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [2]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

def to_obj(x):
    return json.loads(x) if isinstance(x, str) else x

INFO:dapr_agents.llm.openai.client.base:Initializing Azure OpenAI client...
INFO:dapr_agents.llm.openai.client.azure:Using API key for authentication.


In [3]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
    "MCP_JUPYTER_TOOLSETS": "postgresql",
    "DATABASE_URL": os.getenv("DATABASE_URL"),
}

# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="uvx",
    #args=[ "mcp-jupyter-notebook", "--transport", "stdio"],
    args = ["--refresh", "mcp-jupyter-notebook", "--transport", "stdio"],
    #args=[
    #    "--refresh",
    #    "--from",
    #    "../",
    #    "mcp-jupyter-notebook",
    #    "--transport",
    #    "stdio",
    #],
    env=ENV,
)

In [4]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = await AgentTool.from_mcp_session(s)

        from dapr_agents import Agent

        notebook_agent = Agent(
            name="Jupyter Notebook Orchestrator",
            role="A notebook co-pilot that drives analysis inside a persistent Jupyter session, with read-only access to a PostgreSQL database via MCP tools.",
            goal=(
                "Use the active Jupyter notebook to complete tasks end-to-end: "
                "add clear markdown, execute Python code, and, when necessary, install Python packages. "
                "Reuse the same live session (kernel + document) so work builds on previous cells. "
                "When data is needed, discover Postgres schemas/tables first, then load results into DataFrames via the provided tools."
            ),
            instructions=[
                # Session & ordering
                "You are driving a live Jupyter notebook (persistent kernel + document).",
                "Cell order matters. Issue exactly ONE tool call at a time and wait for its result before the next.",
                "Append new cells; do not rewrite or reorder existing cells unless explicitly asked.",

                # Notebook tools
                "Use `notebook.markdown.add(content)` for headings and short explanations.",
                "Use `notebook.code.run(content)` to append and execute Python code cells.",

                # Packages
                "If an import fails, call `notebook.packages.add([pip_names])` once, then retry the import.",
                "Package names are pip distributions (e.g., 'matplotlib', 'plotly', 'requests').",

                # PostgreSQL tools & workflow
                "Before querying data, inspect metadata using schema tools:",
                "• `db.postgresql.schema.list_tables(schema_name=None, include_matviews=False)` to enumerate tables/views.",
                "• `db.postgresql.schema.list_columns(schema_name, table)` to view a table's columns.",
                "• `db.postgresql.schema.tree(limit_per_schema=100)` for a compact overview of schemas → tables.",
                "Only when you actually need rows, call `db.postgresql.query.to_df(raw_sql, limit=50)`.",
                "That tool creates a DataFrame in the kernel and returns `df_name`. Capture and reuse that exact variable name in subsequent `notebook.code.run` cells.",
                "Prefer small, targeted SQL with an explicit LIMIT while exploring.",
                "Never print or echo credentials/DSNs; do not read environment variables into the notebook output.",

                # Notebook hygiene
                "Before each task, add a short markdown heading describing what you'll do.",
                "Keep code cells small and focused—one idea per cell.",
                "When previewing data, show concise outputs (e.g., df.head(), shapes, key metrics).",
                "For Matplotlib plots, include a title and axis labels; add a legend when helpful.",

                # Robustness
                "If an error occurs, add a brief markdown note explaining the fix, then run a corrected code cell.",
                "Reuse variables and results created earlier; reference them directly rather than reloading the same data.",

                # Closure
                "End a logical section with a short markdown summary of what was produced and learned."
            ],
            llm=llm,
            tools=tools,
            max_iterations=20,
        )

        QUESTION = """
A command and control behavior was blocked on host `vnevado-win10r`, which indicated an active infection by malware that could replicate and receive commands from remote attackers. This malware was active, and precautionary measures should be taken to check for residual signs of infection. The process involved had the ID 1332 and ran the command `curl http://vectorsandarrows.com`.

What is the IP address associated with the Manatee Tempest activity group detected in this security incident?
"""
        await notebook_agent.run(QUESTION)

INFO:dapr_agents.tool.executor:Tool registered: Notebook.Markdown.Add
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Code.Run
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Packages.Add
INFO:dapr_agents.tool.executor:Tool registered: Db.Postgresql.Query.ToDf
INFO:dapr_agents.tool.executor:Tool registered: Db.Postgresql.Schema.ListTables
INFO:dapr_agents.tool.executor:Tool registered: Db.Postgresql.Schema.ListColumns
INFO:dapr_agents.tool.executor:Tool registered: Db.Postgresql.Schema.Tree
INFO:dapr_agents.tool.executor:Tool Executor initialized with 7 tool(s).


user:
A command and control behavior was blocked on host `vnevado-win10r`, which indicated an active infection by malware that could replicate and receive commands from remote attackers. This malware was active, and precautionary measures should be taken to check for residual signs of infection. The process involved had the ID 1332 and ran the command `curl http://vectorsandarrows.com`.

What is the IP address associated with the Manatee Tempest activity group detected in this security incident?

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Iteration 1/20 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Calling OpenAI ChatCompletion...


INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
Function name: Db.Postgresql.Schema.ListTables (Call Id: call_aRIbfRi0YOiqZbFpAtNYedyE)
Arguments: {}

--------------------------------------------------------------------------------


...

INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
Function name: Db.Postgresql.Query.ToDf (Call Id: call_bgZnhKimPwjfoLPoRIcUgI5C)
Arguments: {"raw_sql":"SELECT RemoteIP, RemoteUrl, InitiatingProcessId, InitiatingProcessCommandLine, ReportId, TimeGenerated FROM public.\"DeviceNetworkEvents\" WHERE DeviceName = 'vnevado-win10r' AND InitiatingProcessId = '1332' LIMIT 10","limit":10}

--------------------------------------------------------------------------------


INFO:dapr_agents.tool.executor:Running tool (auto): Db.Postgresql.Query.ToDf



Db.Postgresql.Query.ToDf(tool) (Id: call_bgZnhKimPwjfoLPoRIcUgI5C):
{"ok":false,"df_name":"df_db2dcd","create_cell_index":null,"schema":null,"row_count":null,"col_count":null,"sample":null,"error":"Failed to create DataFrame cell","bootstrap":{"ok":true,"error":null}}

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Reached max turns after tool calls; stopping.
INFO:dapr_agents.agents.agent.agent:Agent conversation completed after 20 turns.
